In [1]:
import os
import sys
from pathlib import Path
import numpy as np

import bmi_dpl_model as bmi_model  # This is the BMI we will test.

# Define config path
cfg_file=Path('./conf/model_config.yaml')

if os.path.exists(cfg_file):
    print(" configuration found: " + str(cfg_file))
else:
    print(" no configuration found, exiting...")
    sys.exit()

bmi=bmi_model.BMIdPLModel(cfg_file)

 configuration found: conf/model_config.yaml


In [2]:
bmi.initialize()


KeyError: 'ensemble_type'

In [ ]:
 # debugging ----- #
    n_timesteps, ngrid = dataset_dict['inputs_nn_scaled'].shape
    print(n_timesteps, ngrid)
    # n_timesteps = 400
    # n_basins = 671
    n_basins = dataset_dict['c_nn'].shape[0] 
    print(n_basins)
    # --------------- #

    # Store attributes and forcings in BMI if end_timestep > 100 days.
    # NOTE: maybe data gets passed in this step no matter what, but for <100 day
    # periods, it's only seen by dPL model on update call.
    # n_timesteps = model.config['end_timestep']
    rho = model.config['rho']  # For routing

    i_start = np.arange(0, ngrid, model.config['batch_basins'])
    i_end = np.append(i_start[1:], ngrid)
    
    # if n_timesteps > 100:
    for t in range(len(i_start)):
        # NOTE: for each timestep in this loop, the data assignments below are of
        # arrays of basins. e.g., forcings['key'].shape = (rho + 1, # basins).
        # TODO: add basin batching from pmi.train.py experiment for this loop instead
        # of doing all basins at once.

        # Forward through basins in batches.
        dataset_dict_sample = take_sample_test(model.config, dataset_dict,
                                               i_start[i], i_end[i])
        ################## Map forcings + attributes into BMI ##################
        # Set NN forcings...
        for i, var in enumerate(model.config['observations']['var_t_nn']):
            standard_name = model._var_name_map_short_first[var]
            model.set_value(standard_name, dataset_dict_sample['inputs_nn_scaled'][t:rho + t + 1, :n_basins, i], model='nn')
        n_forc = i
        
        # Set NN attributes...
        for i, var in enumerate(model.config['observations']['var_c_nn']):
            standard_name = model._var_name_map_short_first[var]
            model.set_value(standard_name, dataset_dict_sample['inputs_nn_scaled'][t:rho + t + 1, :n_basins, n_forc + i + 1], model='nn') 

        # Set physics model forcings...
        for i, var in enumerate(model.config['observations']['var_t_hydro_model']):
            standard_name = model._var_name_map_short_first[var]
            model.set_value(standard_name, dataset_dict_sample['x_hydro_model'][t:rho + t + 1, :n_basins, i], model='pm') 

        # Set physics model attributes...
        for i, var in enumerate(model.config['observations']['var_c_hydro_model']):
            standard_name = model._var_name_map_short_first[var]
            # NOTE: These don't have a time dimension.
            model.set_value(standard_name, dataset_dict_samples['c_hydro_model'][:n_basins, i], model='pm')

In [ ]:

        ngrid = dataset_dict['inputs_nn_scaled'].shape[1]
        self.iS = np.arange(0, ngrid, self.config['batch_basins'])
        self.iE = np.append(self.iS[1:], ngrid)

    def _get_model_predictions(self) -> List[Dict[str, torch.Tensor]]:
        """
        Get predictions from a trained model.
        """
        batched_preds_list = []
        for i in tqdm.tqdm(range(len(self.iS)), leave=False, dynamic_ncols=True):
            dataset_dict_sample = take_sample_test(self.config, self.dataset_dict,
                                                   self.iS[i], self.iE[i])
            # Forward pass for hydrology models.
            hydro_preds = self.dplh_model_handler(dataset_dict_sample, eval=True)

            # Compile predictions from each batch.
            if self.config['ensemble_type'] in ['frozen_pnn', 'free_pnn']:
                # For ensembles w/ wNN: Forward pass for wNN to get ensemble weights.
                self.ensemble_lstm(dataset_dict_sample, eval=True)

                # Ensemble hydrology models using learned weights.
                ensemble_pred = self.ensemble_lstm.ensemble_models(hydro_preds)
                batched_preds_list.append({key: tensor.cpu().detach() for key,
                                           tensor in ensemble_pred.items()})
            elif self.config['ensemble_type'] == 'avg':
                # For 'average' type ensemble: Average model predictions at each
                # basin for each day.
                ensemble_pred = model_average(hydro_preds, self.config)
                batched_preds_list.append({key: tensor.cpu().detach() for key,
                                           tensor in ensemble_pred.items()})
            else:
                # For single hydrology model.
                model_name = self.config['hydro_models'][0]
                batched_preds_list.append({key: tensor.cpu().detach() for key,
                                           tensor in hydro_preds[model_name].items()})
        return batched_preds_list

In [ ]:
import torch
import numpy as np
from torch.autograd import Function

class BMIWrapperFunction(Function):
    @staticmethod
    def forward(ctx, input_tensor):
        # Convert PyTorch tensor to NumPy array
        input_np = input_tensor.detach().cpu().numpy()

        # Call the BMI model (assumed to be a method `run_bmi_model`)
        # `run_bmi_model` is a placeholder for your BMI interface
        output_np = run_bmi_model(input_np)  # Replace with actual BMI call

        # Save the input for backward pass
        ctx.save_for_backward(input_tensor)

        # Convert output back to a PyTorch tensor
        output_tensor = torch.from_numpy(output_np).to(input_tensor.device)

        return output_tensor

    @staticmethod
    def backward(ctx, grad_output):
        # Retrieve the saved tensor from forward pass
        input_tensor, = ctx.saved_tensors

        # Convert PyTorch tensor to NumPy array for BMI gradient computation
        grad_output_np = grad_output.detach().cpu().numpy()

        # Compute gradients using BMI model (replace with actual gradient call)
        grad_input_np = compute_bmi_gradient(input_tensor.cpu().numpy(), grad_output_np)

        # Convert gradients back to PyTorch tensor
        grad_input_tensor = torch.from_numpy(grad_input_np).to(input_tensor.device)

        return grad_input_tensor


def run_bmi_model(input_np):
    """Placeholder for running the BMI model."""
    # Your logic for passing data through the BMI interface
    # For example, bmi_model.update(input_np)
    # Return the output NumPy array
    return input_np * 2  # Just an example operation


def compute_bmi_gradient(input_np, grad_output_np):
    """Placeholder for computing BMI model gradients."""
    # Your logic for computing the gradient
    # You may need to implement this manually if BMI doesn't handle it
    return grad_output_np  # Example, assuming identity gradient
